<a href="https://colab.research.google.com/github/GordonWasHere/Electrifying-Tractors-for-Farming-and-Electricity-Grid-Integration/blob/main/tractoroptimisationpassive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Testing Git saving

from google.colab import files;
uploaded = files.upload()

"""
Upload the following files for the code to run:

Currently Used
 - 'Model RF Tractor Use.xlsx'
 - 'Model TOU Cost ActewAGL 2025.xlsx'
 - 'Model Electricity Profile V1.xlsx'


Link to all model files
https://anu365.sharepoint.com/:f:/r/sites/2025ENGN4300CapstoneProjectRepository/Project%20Documents/06.%20Deliverables%20and%20Models/Models%20and%20Data/FINAL%20DATA%20FOR%20IMPLEMENTATION?csf=1&web=1&e=79O5do
"""

Saving Model TOU Cost ActewAGL 2025.xlsx to Model TOU Cost ActewAGL 2025.xlsx


"\nUpload the following files for the code to run:\n\nCurrently Used\n - 'Model RF Tractor Use.xlsx'\n - 'Model TOU Cost ActewAGL 2025.xlsx'\n - 'Model Electricity Profile V1.xlsx'\n\n\nLink to all model files\nhttps://anu365.sharepoint.com/:f:/r/sites/2025ENGN4300CapstoneProjectRepository/Project%20Documents/06.%20Deliverables%20and%20Models/Models%20and%20Data/FINAL%20DATA%20FOR%20IMPLEMENTATION?csf=1&web=1&e=79O5do\n"

In [ ]:
"""
Passive - No Additional Infrastructure Scenario:

Rules:
- The tractor is charged between 10pm to 7am everyday and can be used 7am to 10pm
- The tractor is used as a battery source for residential energy arbitrage.
- The tractor is not used as a battery on days it will be used

Revenue Generation:
- Revenue is generated through residential price arbitrage, storing electricity at low price and using stored energy at high price

Misc Notes:
- Using ActewAGL Energy Plan 2 (Home_Time_Of_Use_Plan_Price_Per_kWh), Column D
- Tractor Battery Capacity is 100kWh, max discharge 14.28 kW, max charging rate 22 kW (Fendt model)
"""

import pandas as pd

# Importing Data ---------------------------------------------------------------

# Tractor Use (Options A -> D)
# Slashing and Undervine Spraying can be done in 1 day or split over 2 days
# 1 day Slashing and Undervine Spraying
excel_tractor_use_A = pd.read_excel("Model RF Tractor Use.xlsx", sheet_name =
                                    'Tractor Load A', usecols = 'I', header=1)
# 1 day Slashing and 2 days Undervine Spraying
excel_tractor_use_B = pd.read_excel("Model RF Tractor Use.xlsx", sheet_name =
                                    'Tractor Load B', usecols = 'I', header=1)
# 2 days Slashing and 1 day Undervine Spraying
excel_tractor_use_C = pd.read_excel("Model RF Tractor Use.xlsx", sheet_name =
                                    'Tractor Load C', usecols = 'I', header=1)
# 2 days Slashing and 2 days Undervine Spraying
excel_tractor_use_D = pd.read_excel("Model RF Tractor Use.xlsx", sheet_name =
                                    'Tractor Load D', usecols = 'I', header=1)
# Note: header=1 is used as document has two header rows


# ActewAGL Residential Electricity Costs
# ActewAGL Plan 1 (Home_Plan_Price_Per_kWh)
excel_R_A1 = pd.read_excel('Model TOU Cost ActewAGL 2025.xlsx', usecols='C')
# ActewAGL Energy Plan 2 (Home_Time_Of_Use_Plan_Price_Per_kWh)
excel_R_A2 = pd.read_excel('Model TOU Cost ActewAGL 2025.xlsx', usecols='D')
# ActewAGL Plan 3 (Home_Daytime_Economy_Plan_Price_Per_kWh)
excel_R_A3 = pd.read_excel('Model TOU Cost ActewAGL 2025.xlsx', usecols='E')


# Farm Electricity Profile
excel_ele_profile = pd.read_excel('Model Electricity Profile V1.xlsx', sheet_name='Total Profile' ,usecols='I')




Index(['Tractor Consumption (kWh)'], dtype='object')


In [ ]:
# Data Processing --------------------------------------------------------------
# Convert to lists

# Using Tractor Use A
# Using  ActewAGL Energy Plan 2

tractor_use = excel_tractor_use_A.iloc[:, 0].tolist()
energy_plan = excel_R_A2.iloc[:, 0].tolist()
ele_profile = excel_ele_profile.iloc[:, 0].tolist()

# Check sum = 8760
if len(tractor_use) != 8760 or len(energy_plan) != 8760 or len(ele_profile) != 8760:
  print(len(tractor_use))
  print(len(energy_plan))
  print(len(ele_profile))
  raise Exception(f"The length of all lists must be 8760 which is the number of hours in a year.")



8760
8760
8760


In [ ]:
"""

Rules:
- The tractor is charged between 10pm to 7am everyday and can be used 7am to 10pm
- Mathematically, the tractor is charged on the 0 to 6 and 22 to 23 hours mod 24
- The tractor is used as a battery source for residential energy arbitrage.
- The tractor is not used as a battery on days it will be used

- Tractor Battery Capacity is 100kWh, max discharge 14.28 kW, max charging rate 22 kW (Fendt model)

"""
# Scenario Implementation ------------------------------------------------------

battery_status = []
capex = 0
arbitrage_revenue = 0


"""

For each hour of the year:

Add charging rate of 22kWh per hour between hours of 0 to 6 and 22 to 23 mod 24 up to the max capacity
Subtract the cost of electricity for each kWh

if tractor is used in the next day:
  set next 24 hours arbitrage to 0
  subtract the electricity used from capacity
  subtract the cost of electricity from the revenue

else
  use the tractor battery subject to the farm profile up to max discharge rate and capacity
  add the equivalent amount it would have costed to revenue

"""